In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing Necessary Libraries

In [ ]:
import os
from PIL import Image
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision.models import resnet50, ResNet50_Weights

### Dataset Loading and Splitting

In [ ]:
data_dir = "/kaggle/input/imagesoasis/Data"  # Original Dataset
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

all_data = []
all_labels = []

for idx, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    for file_name in os.listdir(class_dir):
        if file_name.endswith(".jpg") or file_name.endswith(".png"):
            all_data.append(os.path.join(class_dir, file_name))
            all_labels.append(idx)


train_data, val_data, train_labels, val_labels = train_test_split(
    all_data, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

print(f"Train size: {len(train_data)}, Validation size: {len(val_data)}")

### Class Weights Calculation

In [ ]:
class_counts = [train_labels.count(idx) for idx in range(len(classes))]
class_weights = [1.0 / count for count in class_counts]
class_weights = torch.tensor(class_weights).to("cuda" if torch.cuda.is_available() else "cpu")
print(f"Class counts: {class_counts}")
print(f"Class weights: {class_weights}")

### DementiaDataset Class
- **Purpose**: Custom dataset class for loading image data and labels.

#### Methods:
1. **`__init__`**:
   - Initializes the dataset with image paths (`data`), labels (`labels`), and optional transformations (`transform`).

2. **`__len__`**:
   - Returns the total number of samples in the dataset.

3. **`__getitem__`**:
   - Fetches an image and its corresponding label based on the index (`idx`).
   - Applies the specified transformations to the image if provided.
   - Returns the processed image and its label.


In [ ]:
class DementiaDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label



transform = transforms.Compose([
    transforms.Resize((496, 248)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = DementiaDataset(train_data, train_labels, transform=transform)
val_dataset = DementiaDataset(val_data, val_labels, transform=transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

### Model Initialization and Configuration

In [ ]:
weights = ResNet50_Weights.IMAGENET1K_V1
resnet_model = resnet50(weights=weights)
num_classes = len(classes)
in_features = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model = resnet_model.to(device)


criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(resnet_model.parameters(), lr=0.001)


### Model Training and Validation

In [ ]:
num_epochs = 10
save_model_path = "/kaggle/working/resnet_alzheimer_final.pth"  

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # Train
    resnet_model.train()
    train_loss = 0.0
    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = resnet_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)
    
    # Validation
    resnet_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = resnet_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_val_loss = val_loss / len(val_loader)
    accuracy = 100 * correct / total
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {accuracy:.2f}%")
    
    # Saving the model
    if epoch == num_epochs - 1:
        torch.save(resnet_model.state_dict(), save_model_path)
        print(f"Model saved at last epoch: {save_model_path}")

print("Eğitim Tamamlandı!")


### Model Prediction and Visualization

In [ ]:
model_path = "/kaggle/working/resnet_alzheimer_final.pth"
weights = ResNet50_Weights.IMAGENET1K_V1
resnet_model = resnet50(weights=weights)
num_classes = 4
in_features = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model.load_state_dict(torch.load(model_path, map_location=device))
resnet_model = resnet_model.to(device)
resnet_model.eval()

# Image transforms
transform = transforms.Compose([
    transforms.Resize((496, 248)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Class names
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]


image_paths = {
    "Mild Dementia": "/kaggle/input/imagesoasis/Data/Mild Dementia/OAS1_0031_MR1_mpr-3_147.jpg",
    "Moderate Dementia": "/kaggle/input/imagesoasis/Data/Moderate Dementia/OAS1_0308_MR1_mpr-2_132.jpg",
    "Non Demented": "/kaggle/input/imagesoasis/Data/Non Demented/OAS1_0379_MR2_mpr-2_112.jpg",
    "Very mild Dementia": "/kaggle/input/imagesoasis/Data/Very mild Dementia/OAS1_0142_MR1_mpr-3_129.jpg"
}


fig, axes = plt.subplots(1, len(classes), figsize=(20, 5))

for idx, (class_name, image_path) in enumerate(image_paths.items()):
    
    image = Image.open(image_path).convert("RGB")
    input_image = transform(image).unsqueeze(0).to(device)
    
    
    with torch.no_grad():
        outputs = resnet_model(input_image)
        _, predicted_class = torch.max(outputs, 1)
    
    
    predicted_label = classes[predicted_class.item()]
    
    
    axes[idx].imshow(image)
    axes[idx].set_title(f"Original: {class_name}\nPredict: {predicted_label}")
    axes[idx].axis("off")

plt.tight_layout()
plt.show()